In [1]:
import os

In [2]:
%pwd

'/home/sudhir/SUDHIR/ROSTOCK_COURSE_WORK/SEMESTER_3/ML_100/Disease_Classification/Disease_Classification/research'

In [3]:
# Change directory
os.chdir("../")

In [4]:
%pwd

'/home/sudhir/SUDHIR/ROSTOCK_COURSE_WORK/SEMESTER_3/ML_100/Disease_Classification/Disease_Classification'

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [27]:
# to read yaml files
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories


In [50]:
"""
update constants/__init__.py
use function from utils.common.py
use config/config.yaml 

"""
# reads yaml files 
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Make sure that yaml files are not empty
        self.config = read_yaml(config_filepath)
        # print(self.config)
        self.params = read_yaml(params_filepath) 
        # print(self.params) 

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # creating variable name, data_ingestion from config.yaml

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [51]:
# Now update components
import os
import urllib.request as request # to download dataset using URL
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size



In [52]:
# How data is taken from source
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [49]:
# Lets create pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e
    

[2023-06-19 10:38:07,139 : INFO : common : yaml file: config/config.yaml loaded successfully ]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/sudhirkumardubey/Disease_Classification/raw/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2023-06-19 10:38:07,141 : INFO : common : yaml file: params.yaml loaded successfully ]
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
[2023-06-19 10:38:07,145 : INFO : common : created directory at: artifacts ]
[2023-06-19 10:38:07,146 : INFO : common : created directory at: artifacts/data_ingestion ]
[2023-06-19 10:38:11,031 : INFO : 4037744573 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
C